In [32]:
import folium
from pyproj import crs
import geopandas as gpd
import matplotlib.pyplot as plt

In [51]:
# create map instance
m = folium.Map(location = [39.95300,-75.16355], zoom_start = 10,
              control_scale = True)

# new change
m


In [38]:
# save map to html file
out_fp = 'base_map.html'
m.save(out_fp)

In [49]:
# change the basemap style to stamen toner
# pass in a custom tile set url
api_key = 'cebee8f834f543ceae176470732a4542'
tilemap_url = 'https://tile.thunderforest.com/spinal-map/{z}/{x}/{y}.png?apikey=%s'% api_key
attribution = 'tile.thunderforest.com'
# m2 = folium.Map(location = [39.95300,-75.16355],tiles = tilemap_url, zoom_start = 10,
#               control_scale = True, prefer_canvas = True)
# m2
m2 = folium.Map(location=[39.95300,-75.16355], tiles=tilemap_url,
                zoom_start=12, control_scale=True, prefer_canvas=True, attr = attribution)

m2

## Adding Layers to The Map

In [56]:
# add a marker to the first map
my_house = [39.94824, -75.16243] 

folium.Marker(location = my_house,
             popup = "Keon's House",
             icon = folium.Icon(color = 'green', icon = 'home')).add_to(m)

m # show map

In [57]:
# add layers - point shp - covid test sites
url_testSites = 'http://data-phl.opendata.arcgis.com/datasets/398ec6ac0b7443babcdd41b40bab3407_0.geojson'
points = gpd.read_file(url_testSites)
points.head()

,OBJECTID,globalid,testing_location_nameoperator,contact_phone_number,testing_location_address,testing_restrictions,drive_thruwalk_up,facility_type,Status,ProviderURL,...,Referral,City,TestingLocation2,Notes,Age,Symptoms,Language_Spoken,translation_services,rapid_testing,geometry
0,3,cd1baf05-d6aa-4d34-9666-3e8b49194d98,Penn Medicine Radnor,(610) 902-2000,250 King of Prussia Rd.,netPat,dt,clinic,active,https://www.pennmedicine.org/for-patients-and-...,...,yes,King of Prussia,None,schedApp,year14,None,English,Yes,None,POINT (-75.35860 40.04270)
1,4,861bb02a-affd-4b1b-bad7-b9b8a6c6fe88,HUP Testing Site,(267) 414 2303,4122 Market St.,None,both,fieldSite,active,https://www.pennmedicine.org/coronavirus/drive...,...,no,Philadelphia,None,schedApp,year14,symptom,English,Yes,None,POINT (-75.20421 39.95736)
2,6,f1b8ff06-4f48-4556-9659-a69f4c39f8e8,Temple Health - Jeanes Hospital,(215) 707-6999,7600 Central Ave.,None,dt,hospital,active,https://www.templehealth.org/2019-novel-corona...,...,yes,Philadelphia,None,schedApp,year18,asymptom,English,Yes,None,POINT (-75.08697 40.06872)
3,7,0bcdaa29-729a-42ae-848b-2a9effc21ef5,Jefferson Health - Center City,1-855-462-5333,132 S. 10th St.,netPat,both,hospital,active,https://hospitals.jefferson.edu/jeffconnect.html,...,no,Philadelphia,Parking lot on the corner of 10th and Sansom,schedApp,None,None,English,Yes,None,POINT (-75.15871 39.94963)
4,10,f04e35e4-1f71-4da7-a0cf-7b5aec0a383c,Trinity Health Mid-Atlantic - Nazareth Hospital,None,2601 Holme Ave.,netPat,wu,clinic,active,https://www.trinityhealthma.org/locations/naza...,...,yes,Philadelphia,None,schedApp,year18,None,English,Yes,None,POINT (-75.04165 40.05901)


In [58]:
# convert to folium GeoJson
points_gjson = folium.features.GeoJson(points, name = 'COVID-19 Test Sites')

# add points to the map
points_gjson.add_to(m)

m

## Layer Control

In [59]:
# create a layer control object and add it to our map
folium.LayerControl().add_to(m)
m

## Heatmap
Make a heatmap - wrangle the data first

In [68]:
from folium.plugins import HeatMap

hm = folium.Map(location = [39.95300,-75.16355], zoom_start = 10,
              control_scale = True)

# get x y coords for each point
points['x'] = points['geometry'].x
points['y'] = points['geometry'].y

# create a list of coordinate pairs
locations = list(zip(points['y'], points['x']))
# print(f'Number of Test Sites: {len(locations)}')

HeatMap(locations).add_to(hm)
hm

In [69]:
# Use marker clusters
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster(locations)

# define a new map instance
cm = folium.Map(location = [39.95300,-75.16355], zoom_start = 10,
              control_scale = True)

# add markers to map
marker_cluster.add_to(cm)

cm

## Chloropleth Maps - Sanitation Maps of Philly

In [75]:
# read in another wfs service 
import requests
import geojson

# fetch data from WFS using requests
sanitation_areas = 'http://data.phl.opendata.arcgis.com/datasets/472c504f650242f4be612d8320b89c86_0.geojson'
data = gpd.read_file(sanitation_areas)
# create a geodataframe from the geojson

# check the data
data.head()


,OBJECTID,SANAREA,Shape__Area,Shape__Length,geometry
0,1,1,7.325642e+08,176216.820105,"POLYGON ((-75.19054 39.96953, -75.19117 39.970..."
1,2,2,5.220482e+08,98308.941469,"POLYGON ((-75.15554 39.95693, -75.15713 39.957..."
2,3,3,3.765106e+08,89914.123213,"POLYGON ((-75.15010 40.01458, -75.15007 40.014..."
3,4,4,6.878803e+08,106443.752566,"POLYGON ((-75.19083 40.07412, -75.19501 40.076..."
4,5,5,4.848414e+08,109513.939056,"POLYGON ((-75.10631 40.03431, -75.10759 40.035..."


In [83]:
# reproject to WGS84 if needed (folium uses web mercator)
# print(str(data.crs))
if str(data.crs) == 'epsg:4326':
    print('The data is in the right projection')
else:
    print('The data should be reprojected')

The data is in the right projection


In [85]:
# rename the sanitation area col
data = data.rename(columns = {'SANAREA': 'Sanitation Area'})

# make a new col for geoID - data needs to have unique identifier index
data['geoid'] = data.index.astype(str)


In [107]:
# create map instance
chm = folium.Map(location = [39.95300,-75.16355], zoom_start = 11,
              control_scale = True, tiles = 'cartodbpositron')
# plot choropleth map - geoid col needs to be assigned as the first col
folium.Choropleth(geo_data = data, data = data, 
                columns = ['geoid', 'Sanitation Area'],
                 key_on = 'feature.id',
                 fill_color = 'YlOrRd',
                highlight = True,
                   legend_name = 'Sanitation Areas In Philadelphia (Source: Open Philly Data)' , 
                 line_weight = 0).add_to(chm)

chm

## Tooltips
Add tooltips, we can add tooltips to our map when plotting the polygons as GeoJson objects using the GeoJsonTooltip feature.

In [110]:
# Convert points to GeoJson
folium.features.GeoJson(data,  
                        name='Labels',
                        style_function=lambda x: {'color':'transparent','fillColor':'transparent','weight':0},
                        tooltip=folium.features.GeoJsonTooltip(fields=['Sanitation Area'],
                                                                aliases = ['Sanitation Area:'],
                                                                labels=True,
                                                                sticky=False
                                                                            )
                       ).add_to(chm)
chm


In [111]:
outfp = "choropleth_map.html"
chm.save(outfp)